In [4]:
from spatialoperations import SpatialOperations
from spatialoperations.config import SpatialOpsConfig


config = SpatialOpsConfig(
    root="s3://cl-lake-test/sids-test-tough-topobathy",
    dx=0.00026977476123492776,
    epsg=4326,
    bounds=[-179.99, -50, 179.99, 50],
    chunk_size=5000,
)
so = SpatialOperations(config=config)
so

In [3]:
# so.ro.zarr_manager.root_group.create_group("topobathy/merged")
ds = so.ro.intake.get_data_layout(["FABDEM_ACA_merged", "FABDEM_ACA_merged_gapfilled"])
so.ro.intake.write_dataset_schema(ds, "topobathy/merged", overwrite_kwargs={"mode": "w"})

In [5]:
import geopandas as gpd
import numpy as np
from tqdm.notebook import tqdm
from spatialoperations.types.geotypes import Bounds

countries = [
    "FSM_10",
    # "FSM_05",
    # "FSM_08",
    # "FSM_21",
    # "FSM_20",
    # "FSM_18",
    # "FSM_11",
    # "KIR_09"
]
gdf = gpd.read_file("/cccr-lab/001_projects/002_nbs-adapt/003_regions_study_units/All_Regions_Marina/02_Polygons_SIDS_20250701/Polygons_SIDS_20250701_final.gpkg")   
regions = gdf[
    gdf["Subreg_Name"].apply(lambda x: np.any([c in x for c in countries]))
]
regions['idxs'] = regions.geometry\
    .apply(lambda x: Bounds(*x.bounds, crs=4326))\
    .apply(lambda x: so.ro.geobox_manager.idxs_for_bounds(x))
regions




/opt/conda/lib/python3.12/site-packages/geopandas/geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,region_id,HAS_REEFS,NumberOfCe,PolygonAre,ACADataAva,ACABenthic,ACABathyme,utmzone,utmcode,Subreg_Name,geometry,idxs
52,53,NaN,4.193852e+06,3.774467e+09,YES,YES,YES,56PN,32656,FSM_10,"MULTIPOLYGON (((150.21069 9.05281, 150.53484 8...","[(30, 244), (30, 245)]"


In [8]:
!micromamba install -y -n base -c conda-forge scikit-image

[+] 0.0s
[+] 0.1s
conda-forge/linux-64 ━━━━━━━╸━━━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.0s
conda-forge/noarch   ━━━━━━╸━━━━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.0s[+] 0.2s
conda-forge/linux-64 ╸━━━━━━━━━━━━━━━━━━━━━━   3.3MB /  45.3MB @  20.4MB/s  0.1s
conda-forge/noarch   ━━━━━━━━━━━━━━━━━━━━━━━  68.7kB /  21.4MB @ 425.3kB/s  0.1s[+] 0.3s
conda-forge/linux-64 ━━╸━━━━━━━━━━━━━━━━━━━━   7.4MB /  45.3MB @  28.1MB/s  0.2s
conda-forge/noarch   ━╸━━━━━━━━━━━━━━━━━━━━━   2.0MB /  21.4MB @   7.7MB/s  0.2s[+] 0.4s
conda-forge/linux-64 ━━━━╸━━━━━━━━━━━━━━━━━━  11.7MB /  45.3MB @  32.0MB/s  0.3s
conda-forge/noarch   ━━━╸━━━━━━━━━━━━━━━━━━━   4.2MB /  21.4MB @  11.4MB/s  0.3s[+] 0.5s
conda-forge/linux-64 ━━━━━━━╸━━━━━━━━━━━━━━━  16.9MB /  45.3MB @  36.2MB/s  0.4s
conda-forge/noarch   ━━━━━━╸━━━━━━━━━━━━━━━━   6.7MB /  21.4MB @  14.2MB/s  0.4s[+] 0.6s
conda-forge/linux-64 ━━━━━━━━━━╸━━━━━━━━━━━━  22.1MB /  45.3MB @  38.9MB/s  0.5s
conda-forge/noarch   ━━━━━━━━╸━━━━━━━━━━━━━━   8.7M

In [ ]:
from spatialoperations.rasterops.export import Export
from spatialoperations.compute import JoblibCompute
from spatialoperations.rasterops.compilers import Compilers
import os, shutil
from tqdm.notebook import tqdm
import skimage
import numpy as np
from datetime import datetime

ACA_MIN = [-20]
GEBCO_MAX = [-100]
OBJECT_SIZE_MIN = [50]
GAPFILL_METHODS = ["linear"]

ACA_MAX = -0.2
COMPRESS = "zstd"
# ACA_MIN = [-10, -15, -20]
# GEBCO_MAX = [-200, -100, -50]
# OBJECT_SIZE_MIN = [10, 100, 1000]


RESTART = False

jlc = JoblibCompute(n_jobs=20)
exporter = Export(so.config)
for _index, sample in tqdm(regions.iterrows(), desc="Regions"):
    print(f"Processing {sample.Subreg_Name}: {len(sample.idxs)}")
    output_dir = f"test/{sample.Subreg_Name}-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    if os.path.exists(output_dir) and RESTART:
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)
    
    aca = exporter.as_da(
        var="ACA",
        group="topobathy/bathymetry",
        idxs=sample.idxs,
        COG=True,
        compute=jlc,
    )
    fabdem = exporter.as_da(
        var="FABDEM",
        group="topobathy/topography",
        idxs=sample.idxs,
        COG=True,
        compute=jlc,
    )
    gebco = exporter.as_da(
        var="GEBCO",
        group="topobathy/bathymetry",
        idxs=sample.idxs,
        COG=True,
        compute=jlc,
    )
    aca = aca.where(aca < ACA_MAX)
    fabdem_mask = fabdem.where(fabdem > 0)
    
    for var, da in [("ACA", aca), ("FABDEM", fabdem), ("GEBCO", gebco)]:
        print(f"Exporting {var}")
        da.rio.to_raster(
            f"{output_dir}/{var}.tif",
            compress="deflate",
        )
        
    for aca_min in ACA_MIN:
        print(f"Processing ACA_MIN={aca_min}")
        aca_test = aca.where(aca > aca_min)
        aca_test.rio.to_raster(
            f"{output_dir}/ACA-gt-{aca_min}.tif",
            compress=COMPRESS,
        )
        print(f"Processing small objects removed with ACA_MIN={aca_min}")
        aca_small_objects_removed_dict = dict()
        for k in OBJECT_SIZE_MIN:
            aca_small_objects_removed_dict[k] = aca_test.where(
                skimage.morphology.remove_small_objects(
                    ~np.isnan(aca_test.data),
                    min_size=k,
                )
            )
        for k, v in aca_small_objects_removed_dict.items():
            v.rio.to_raster(
                f"{output_dir}/ACA-gt-{aca_min}_smallobjectsremoved-lt{k}.tif",
                compress=COMPRESS,
            )
        for gebco_max in GEBCO_MAX:
            fid = f"ACA-gt-{aca_min}_GEBCO-lt-{gebco_max}"
            print(f"Processing merge with {fid}")
            gebco_test = gebco.where(gebco < gebco_max)
            gebco_test.rio.to_raster(
                f"{output_dir}/GEBCO-lt-{gebco_max}.tif",
                compress=COMPRESS,
            )
            merged = aca_test.copy(data=Compilers.compile_max.value(np.array([aca_test, fabdem, gebco_test])))
            merged.rio.to_raster(
                f"{output_dir}/merged_{fid}_withgaps.tif",
                compress=COMPRESS,
            )
            merged_small_objects_removed_dict = dict()
            print(f"Processing merge with small objects removed with {fid}")
            for k, v in aca_small_objects_removed_dict.items():
                merged_small_objects_removed = aca.copy(data=Compilers.compile_max.value(np.array([v, fabdem, gebco_test])))
                merged_small_objects_removed.rio.to_raster(
                    f"{output_dir}/merged_{fid}_smallobjectsremoved-lt{k}.tif",
                    compress=COMPRESS,
                )
                merged_small_objects_removed_dict[k] = merged_small_objects_removed
                
            merged = merged.rio.clip_box(*sample.geometry.bounds)
            print(f"Processing gapfilled with {fid}")
            for method in GAPFILL_METHODS:
                gapfilled = merged.rio.interpolate_na(method=method)
                gapfilled.rio.to_raster(
                    f"{output_dir}/merged_{fid}_gapfilled_{method}.tif",
                    compress=COMPRESS,
                )
                print(f"Processing gapfilled with {fid} and small objects removed")
                for k, v in merged_small_objects_removed_dict.items():
                    gapfilled_small_objects_removed = v.rio.interpolate_na(method=method)
                    gapfilled_small_objects_removed.rio.to_raster(
                        f"{output_dir}/merged_{fid}_gapfilled_{method}_smallobjectsremoved-lt{k}.tif",
                        compress=COMPRESS,
                    )


Regions: 0it [00:00, ?it/s]

Processing FSM_10: 2


/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:330: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
Warning 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/5482
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/5936
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/3978
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/2193


Export progress: 0.0% complete
Export progress: 1.5% complete
Export progress: 2.3% complete
Export progress: 3.1% complete
Export progress: 4.6% complete
Export progress: 5.4% complete
Export progress: 6.2% complete
Export progress: 7.7% complete
Export progress: 8.5% complete
Export progress: 9.2% complete
Export progress: 10.8% complete
Export progress: 11.5% complete
Export progress: 12.3% complete
Export progress: 13.1% complete
Export progress: 14.6% complete
Export progress: 15.0% complete
Export progress: 16.2% complete
Export progress: 17.3% complete
Export progress: 18.1% complete
Export progress: 19.2% complete
Export progress: 20.0% complete


ERROR 1: PROJ: proj_create_from_name: Open of /opt/conda/share/proj failed


...

Export progress: 21.1% complete
Export progress: 22.2% complete
Export progress: 23.2% complete
Export progress: 24.2% complete
Export progress: 25.3% complete
Export progress: 26.2% complete
Export progress: 27.1% complete
Export progress: 28.3% complete
Export progress: 29.2% complete
Export progress: 30.1% complete
Export progress: 31.3% complete
Export progress: 32.2% complete
Export progress: 33.1% complete
Export progress: 34.3% complete
Export progress: 35.2% complete
Export progress: 36.1% complete
Export progress: 37.3% complete
Export progress: 38.2% complete
Export progress: 39.1% complete
Export progress: 40.3% complete
Export progress: 41.2% complete
Export progress: 42.1% complete
Export progress: 43.3% complete
Export progress: 44.2% complete
Export progress: 45.1% complete
Export progress: 46.3% complete
Export progress: 47.2% complete
Export progress: 48.1% complete
Export progress: 49.3% complete
Export progress: 50.2% complete
Export progress: 51.1% complete
Export p

Warning 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


Export progress: 0.0% complete
Export progress: 1.5% complete
Export progress: 2.3% complete
Export progress: 3.1% complete
Export progress: 4.6% complete
Export progress: 5.4% complete
Export progress: 6.2% complete
Export progress: 7.7% complete
Export progress: 8.5% complete
Export progress: 9.2% complete
Export progress: 10.8% complete
Export progress: 11.5% complete
Export progress: 12.3% complete
Export progress: 13.1% complete
Export progress: 14.6% complete
Export progress: 15.0% complete
Export progress: 16.2% complete
Export progress: 17.3% complete
Export progress: 18.1% complete
Export progress: 19.2% complete
Export progress: 20.0% complete


ERROR 1: PROJ: proj_create_from_name: Open of /opt/conda/share/proj failed


Export progress: 21.1% complete
Export progress: 22.2% complete
Export progress: 23.2% complete
Export progress: 24.2% complete
Export progress: 25.3% complete
Export progress: 26.2% complete
Export progress: 27.1% complete
Export progress: 28.3% complete
Export progress: 29.2% complete
Export progress: 30.1% complete
Export progress: 31.3% complete
Export progress: 32.2% complete
Export progress: 33.1% complete
Export progress: 34.3% complete
Export progress: 35.2% complete
Export progress: 36.1% complete
Export progress: 37.3% complete
Export progress: 38.2% complete
Export progress: 39.1% complete
Export progress: 40.3% complete
Export progress: 41.2% complete
Export progress: 42.1% complete
Export progress: 43.3% complete
Export progress: 44.2% complete
Export progress: 45.1% complete
Export progress: 46.3% complete
Export progress: 47.2% complete
Export progress: 48.1% complete
Export progress: 49.3% complete
Export progress: 50.2% complete
Export progress: 51.1% complete
Export p